In [0]:
display(spark.sql("USE CATALOG catalog"))
display(spark.sql("USE SCHEMA schema"))

In [0]:
# read the information using a spark dataframe
df = (spark.read
      .option("header","true")
      .option("delimiter",";")
      .csv('/Volumes/tables_datageo.csv')
)
df.limit(5).display()

nom_schema,nom_table,info_table,type_table,size,nombre_colonnes,nombre_lignes
siggaia,rai_pkloc,null,BASE TABLE,223 MB,13,324826
siggaia,rai_sens,null,BASE TABLE,16 MB,24,1708
siggaia,rai_signal,null,BASE TABLE,198 MB,61,132264
siggaia,rai_troncon_voie,null,BASE TABLE,453 MB,28,36982
siggaia,rai_voie,null,BASE TABLE,33 MB,22,9951


In [0]:
# create a list of the name of schemas and a list of the name of tables, both from the spark dataframe
schemas = [row['nom_schema'] for row in df.select('nom_schema').collect()]
tables = [row['nom_table'] for row in df.select('nom_table').collect()]

In [0]:
# set a connection using jdbc and spark

jdbc_url = "jdbc:postgresql://host:port/db"

for schema, table in zip(schemas, tables):
    dbtable = f"{schema}.{table}"
    df = spark.read.format("jdbc").options(
        url=jdbc_url,
        dbtable=dbtable,
        user="user",
        password="password",
        driver="org.postgresql.Driver"
    ).load()
    
    df.write.saveAsTable(f"`{schema}-{table}`")